In [2]:
import os
import pydicom
import cv2
from tensorflow.keras.applications.resnet_v2 import preprocess_input

2024-04-21 00:51:40.490812: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-21 00:51:40.543816: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-21 00:51:42.208760: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-21 00:51:43.660329: I itex/core/wrapper/itex_cpu_wrapper.cc:42] Intel Extension for Tensorflow* AVX512 CPU backend is loaded.
2024-04-21 00:51:44.315926: I itex/core/wrapper/itex_gpu

In [3]:
import pandas as pd
import numpy as np
import keras
from keras import layers
import keras
from keras.applications import ResNet50V2
from keras.applications.resnet50 import preprocess_input, decode_predictions

## Preprocessing the data so that ResNet can accept the images

In [8]:
def load_preprocessed_dicom_files(directory):
    images = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.dcm'):
                file_path = os.path.join(root, file)
                image = load_and_preprocess_dicom(file_path)
                images.append(image)
    return np.array(images)

In [9]:
#loading the  input data from path
X_train_path = 'images_train'
X_test_path = 'images_test'
X_valid_path = 'images_valid'

train_X = load_preprocessed_dicom_files(X_train_path)
valid_X = load_preprocessed_dicom_files(X_valid_path)
test_X  = load_preprocessed_dicom_files(X_test_path)

In [12]:
labels_csv = 'train.csv'
labels_df = pd.read_csv(labels_csv)
labels_df = labels_df.iloc[:, [2, 3]]
labels_df

,SOPInstanceUID,pe_present_on_image
0,c0f3cb036d06,0
1,f57ffd3883b6,0
2,41220fda34a3,0
3,13b685b4b14f,0
4,be0b7524ffb4,0
...,...,...
1790589,da0ecef50cf5,0
1790590,d74b46c2f2c4,0
1790591,ba71189191ad,0
1790592,f4fdc88f2ace,0


 ### Here, we instantiate a base model with pre-trained weights.

In [6]:
base_model = keras.applications.ResNet50V2(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

2024-04-20 21:49:39.829928: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform XPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-20 21:49:39.829976: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:XPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: XPU, pci bus id: <undefined>)


### Then, we freeze the base model.


In [7]:
base_model.trainable = False

### Create a new model on top of the pretrained model

In [8]:
inputs = keras.Input(shape=(150, 150, 3))
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

### Training the new model on blood clot data

In [9]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    min_delta=0.00001, # minimium amount of change to count as an improvement
    patience= 30, # how many epochs to wait before stopping
    restore_best_weights=True,
)

In [10]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

### Plotting the loss history

In [ ]:
history_df = pd.DataFrame(history.history)#this creates a new df that keeps track of the training/validation 
history_df.loc[:, ['loss', 'val_loss']].plot();#this is used to visualize the loss history

print("Minimum validation loss: {}".format(history_df['val_loss'].min()))
print("Minimum training loss: {}".format(history_df['loss'].min()))

## ! fix  X_train, y_train, X_valid,y_valid

In [ ]:
# Unfreeze the base model
base_model.trainable = True

# It's important to recompile your model after you make any changes
# to the `trainable` attribute of any inner layer, so that your changes
# are take into account
model.compile(optimizer=keras.optimizers.Adam(1e-5),  # Very low learning rate
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

# Train end-to-end. Be careful to stop before you overfit!
history = model.fit(
    X_train, y_train, #ADD the CORRECT df's for testing data
    validation_data=(X_valid,y_valid), #ADD the CORRECT df's for validation data
    batch_size=64,
    epochs=500,
    callbacks=[early_stopping], # put callbacks in a list
    verbose=0,  # turn off training log
)